# Notes

## Day and short-term strategies

- Scalping: Taking many very small intraday moves, aiming for a few cents or ticks per trade, often with large size and high frequency.
- Momentum trading: Buying strength (or shorting weakness) and “riding the wave” as price and volume accelerate in one direction.
- Range trading: Buying near support and selling near resistance when an asset is oscillating in a well-defined price range.
- Breakout trading: Entering when price pushes through a key support/resistance level or consolidation range, expecting a strong follow-through move.
- News-based trading: Reacting to catalysts like earnings, economic data, or company headlines that cause sudden volatility and volume spikes.[1][3][4][5][7]

## What EMA means in trading terms

- It's a smoothed line on the chart that represents the average price over the last N periods, with recent candles influencing it more than older ones.  
- A “20 EMA” on a 5-minute chart = the exponentially weighted average of the last 20 five-minute closes, updating each bar.  
- Traders use it to:
  - Identify trend direction (price above a rising EMA = uptrend bias, below a falling EMA = downtrend bias).  
  - Act as “dynamic support/resistance” for pullback entries (buy near a rising EMA, sell near a fallin

  So when you see code like `df["ema20"]`, that column is just “the 20-period exponential moving average of price,” used as a faster, more responsive trend line than an SMA.

## Volume Impact

Volume refers to the number of shares or contracts traded for a particular stock during a given period, such as a minute, hour, or day. In day trading, volume is a key indicator showing the strength or interest in a stock's price movement.

### Importance of Volume in Day Trading

- **Liquidity:** Higher volume usually means the stock is easier to buy and sell quickly, making it more suitable for day trading.
- **Confirmation:** Traders often look for high volume to confirm the strength of a price trend or pattern. For example, a breakout above resistance is considered more reliable if it's accompanied by increased volume.
- **Volatility:** Surges in volume can signal heightened interest and potentially greater price swings, which create more opportunities for quick profits—or losses.

### Popular Strategies Incorporating Volume

- Volume Breakouts: Traders buy when a stock breaks above a price resistance on high volume, expecting the momentum to continue.
- Volume Reversals: Sudden spikes in volume after a prolonged trend may signal a reversal or exhaustion, prompting traders to take the opposite position.
- Volume Oscillators: Some strategies use indicators like the Volume Weighted Average Price (VWAP) or On Balance Volume (OBV) to find buy or sell signals based on volume trends.

Volume provides insights into crowd behavior and market dynamics, making it one of the most watched metrics by day traders. By analyzing volume patterns, traders aim to improve timing and confidence in their trades.

## Walk-forward analysis vs back testing

A simulated live trading test on historical data (often called walk-forward analysis, market replay, or bar replay simulation) mimics real-time trading by revealing data gradually as if you're trading live, rather than seeing the full history upfront.​

How it works (vs regular backtest)

| Regular Backtest                          | Simulated Live Trading                                         |
| ----------------------------------------- | -------------------------------------------------------------- |
| Sees all data at once; knows future prices | Data revealed bar-by-bar; can't peek ahead                     |
| Tests full dataset immediately            | Replay mode: play/pause/rewind historical bars at chosen speed |
| Fast but can overfit                      | Slower but realisticdecision-making under time pressure        |

# Code

In [11]:
import yfinance as yf
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime
import numpy as np

In [12]:
# sofi = yf.Ticker("SOFI")

In [ ]:
# hist = sofi.history(period="8d", interval="1m")
tickers_df = yf.download("SOFI", period="1d", interval="5m").tz_convert("America/New_York")
tickers_df.dropna(inplace=True)
"""
# Example: NYSE calendar (SOFI)
nyse = mcal.get_calendar("NYSE")

# Get all valid trading days over your data range
schedule = nyse.schedule(start_date=hist.index.min().date(),
                         end_date=hist.index.max().date())
trading_days = schedule.index  # DatetimeIndex of trading days
mask = hist.index.tz_convert(None).normalize().isin(trading_days)
print(trading_days)
print(mask)
# Keep only rows whose date is a trading day
hist = hist[mask]
"""

sofi_df = tickers_df.xs("SOFI", level="Ticker", axis=1)
sofi_number_index_df = sofi_df.reset_index()
sofi_df, sofi_number_index_df

/tmp/ipykernel_28163/3773341151.py:2: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


(Price                          Close       High        Low       Open   Volume
 Datetime                                                                      
 2025-11-24 09:30:00-05:00  26.309999  26.400000  25.733900  25.820000  2954507
 2025-11-24 09:35:00-05:00  26.090000  26.420000  26.020000  26.309999  1342992
 2025-11-24 09:40:00-05:00  25.940001  26.315001  25.924999  26.094999  1173139
 2025-11-24 09:45:00-05:00  25.840000  26.120001  25.810900  25.940001  1024676
 2025-11-24 09:50:00-05:00  26.049999  26.189899  25.655001  25.840000  1095291
 ...                              ...        ...        ...        ...      ...
 2025-12-04 15:35:00-05:00  29.410000  29.440001  29.360001  29.410101   263830
 2025-12-04 15:40:00-05:00  29.350000  29.410000  29.340000  29.405001   231139
 2025-12-04 15:45:00-05:00  29.440001  29.459999  29.320000  29.350000   430890
 2025-12-04 15:50:00-05:00  29.547701  29.575399  29.434999  29.440001  1135137
 2025-12-04 15:55:00-05:00  29.580000  2

In [14]:
fig = go.Figure(data=[go.Candlestick(x=sofi_number_index_df['Datetime'],
                open=sofi_number_index_df['Open'],
                high=sofi_number_index_df['High'],
                low=sofi_number_index_df['Low'],
                close=sofi_number_index_df['Close'])])

fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]),                # hide weekends
        dict(bounds=[16, 9.5], pattern="hour"),     # hide 4pm–9:30am

    ]
)
fig.update_layout(
    xaxis_rangeslider_visible=False,
    height=800
)
fig.show()

## Momentum Strategy

### Momentum long signal: rising EMA + high volume

In [15]:
# Indicators
sofi_ema20_high_volume_df = sofi_df.copy()
sofi_ema20_high_volume_df["ema20"] = sofi_ema20_high_volume_df["Close"].ewm(span=20, adjust=False).mean()
sofi_ema20_high_volume_df["ema20_prev"] = sofi_ema20_high_volume_df["ema20"].shift(1)
sofi_ema20_high_volume_df["ema_rising"] = sofi_ema20_high_volume_df["ema20"] > sofi_ema20_high_volume_df["ema20_prev"]

sofi_ema20_high_volume_df["vol_ma20"] = sofi_ema20_high_volume_df["Volume"].rolling(20).mean()
sofi_ema20_high_volume_df["high_vol"] = sofi_ema20_high_volume_df["Volume"] > sofi_ema20_high_volume_df["vol_ma20"]

# Momentum long signal: price above rising EMA + high volume
sofi_ema20_high_volume_df["long_signal"] = (sofi_ema20_high_volume_df["Close"] > sofi_ema20_high_volume_df["ema20"]) & sofi_ema20_high_volume_df["ema_rising"] & sofi_ema20_high_volume_df["high_vol"]

# Backtest (very simplified, 1 position max, no fees/slippage)
capital = 10_000
position = 0
entry_price = 0.0
pnl = []

for i in range(1, len(sofi_ema20_high_volume_df)):
    t = sofi_ema20_high_volume_df.index[i]
    row = sofi_ema20_high_volume_df.iloc[i]
    prev_row = sofi_ema20_high_volume_df.iloc[i-1]

    # Flatten before close to avoid overnight
    if position != 0 and t.time() >= pd.to_datetime("15:55").time():
        pnl.append((row["Close"] - entry_price) * position)
        position = 0
        entry_price = 0.0
        continue

    # If flat and signal, enter
    if position == 0 and row["long_signal"]:
        entry_price = row["Close"]
        position = capital // entry_price  # all-in sizing
        continue

    # If in position, check TP / SL or exit conditions
    if position != 0:
        # take profit 1%
        if row["High"] >= entry_price * 1.01:
            exit_price = entry_price * 1.01
            pnl.append((exit_price - entry_price) * position)
            position = 0
            entry_price = 0.0
            continue

        # stop loss 0.5%
        if row["Low"] <= entry_price * 0.995:
            exit_price = entry_price * 0.995
            pnl.append((exit_price - entry_price) * position)
            position = 0
            entry_price = 0.0
            continue

        # momentum breakdown: close below EMA20
        if row["Close"] < row["ema20"]:
            exit_price = row["Close"]
            pnl.append((exit_price - entry_price) * position)
            position = 0
            entry_price = 0.0

total_pnl = sum(pnl)
print("Total PnL:", total_pnl)


Total PnL: 251.51162480354446


### Golden Ratio Identifier

In [16]:
def find_swings(df, window=5):
    """Detect swing highs/lows using pivot points"""
    highs = df['High'].rolling(window*2+1, center=True).apply(lambda x: x.iloc[window] if x.iloc[window] == x.max() else np.nan)
    lows = df['Low'].rolling(window*2+1, center=True).apply(lambda x: x.iloc[window] if x.iloc[window] == x.min() else np.nan)
    return highs.dropna(), lows.dropna()

def golden_pocket(df, window=5):
    """Calculate latest Golden Pocket zone"""
    highs, lows = find_swings(df, window)
    
    # Get most recent swing high and low
    recent_high = highs.iloc[-1] if len(highs) > 0 else df['High'].max()
    recent_low = lows.iloc[-1] if len(lows) > 0 else df['Low'].min()
    
    fib_range = abs(recent_high - recent_low)
    
    # Golden Pocket: 61.8% to 65% retracement from high
    gp_upper = recent_high - (0.618 * fib_range)
    gp_lower = recent_high - (0.65 * fib_range)
    
    return {
        'swing_high': recent_high,
        'swing_low': recent_low,
        'gp_upper': gp_upper,
        'gp_lower': gp_lower,
        'in_zone': (gp_lower <= df['Close'].iloc[-1] <= gp_upper)
    }

In [17]:
# Usage
result = golden_pocket(sofi_ema20_high_volume_df)
print(f"Golden Pocket: {result['gp_lower']:.2f} - {result['gp_upper']:.2f}")
print(f"Current price in zone: {result['in_zone']}")

Golden Pocket: 29.15 - 29.17
Current price in zone: False


In [ ]:
for i in range()